In [12]:
! wget https://developer.nvidia.com/compute/cuda/8.0/Prod2/local_installers/cuda-repo-ubuntu1604-8-0-local-ga2_8.0.61-1_amd64-deb -O cuda-repo-ubuntu1604-8-0-local-ga2_8.0.61-1_amd64.deb
! dpkg -i cuda-repo-ubuntu1604-8-0-local-ga2_8.0.61-1_amd64.deb
! apt-get update
! apt-get install cuda

--2020-01-15 17:35:06--  https://developer.nvidia.com/compute/cuda/8.0/Prod2/local_installers/cuda-repo-ubuntu1604-8-0-local-ga2_8.0.61-1_amd64-deb
Resolving developer.nvidia.com (developer.nvidia.com)... 192.229.189.146
Connecting to developer.nvidia.com (developer.nvidia.com)|192.229.189.146|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://developer.download.nvidia.com/compute/cuda/8.0/secure/Prod2/local_installers/cuda-repo-ubuntu1604-8-0-local-ga2_8.0.61-1_amd64.deb?6JGS3PVViApSNp-vB_SoQ2KG3k6bkBu0I7fs2NkN-hNnF0O6mnRlU3xiPx5AwPnZrRhraNnQPj1J5ZYSBD3YmbQmbp2xrmB86sHnYl8EkCDK3s8Xps2F0ycnLkifxfJx18rSrBjM6a1w-VizXrvYd5S6k2NorFgWDWtapyfQbAvyX0EYDRtXR1ABlxGV2fvr7nEp0iEXznMfqFVJoGe8Ujcanw [following]
--2020-01-15 17:35:08--  https://developer.download.nvidia.com/compute/cuda/8.0/secure/Prod2/local_installers/cuda-repo-ubuntu1604-8-0-local-ga2_8.0.61-1_amd64.deb?6JGS3PVViApSNp-vB_SoQ2KG3k6bkBu0I7fs2NkN-hNnF0O6mnRlU3xiPx5AwPnZrRhraNnQPj1J5ZYSBD3YmbQmbp2x

In [0]:
import numpy as np
import numpy.random as rd

In [0]:
import os
os.environ["THEANO_FLAGS"] = "mode=FAST_RUN,device=cuda,floatX=float32"

In [0]:
import theano
import theano.tensor as T

In [16]:
print(theano.config.device)

cuda


In [0]:
a = T.fvector()
b = T.fvector()

f = theano.function(inputs=[a, b], outputs=[T.dot(a, b)])

In [18]:
f(
    rd.random(size=(1000_000_000)).astype(np.float32), 
    rd.random(size=(1000_000_000)).astype(np.float32)
)

KeyboardInterrupt: ignored

In [19]:
from theano import function, config, shared, tensor
import numpy
import time

vlen = 10 * 30 * 768  # 10 x #cores x # threads per core
iters = 1000

rng = numpy.random.RandomState(22)
x = shared(numpy.asarray(rng.rand(vlen), config.floatX))
f = function([], tensor.exp(x))
print(f.maker.fgraph.toposort())
t0 = time.time()
for i in range(iters):
    r = f()
t1 = time.time()
print("Looping %d times took %f seconds" % (iters, t1 - t0))
print("Result is %s" % (r,))
if numpy.any([isinstance(x.op, tensor.Elemwise) and
              ('Gpu' not in type(x.op).__name__)
              for x in f.maker.fgraph.toposort()]):
    print('Used the cpu')
else:
    print('Used the gpu')

[Elemwise{exp,no_inplace}(<TensorType(float32, vector)>)]
Looping 1000 times took 1.448643 seconds
Result is [1.2317803 1.6187934 1.5227807 ... 2.2077181 2.2996776 1.6232328]
Used the cpu
